# Multivariate linear regression


In this notebook, we will make use of the multivariate linear regression to predict the price of houses. We will use the Boston house price dataset available at https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html for our purpose. The dataset includes 14 attributes per house and its price. Some examples of the attributes are:

|  Attribute       | Definition    | 
| :-------------:  | :-------------:| 
| CRIM             | per capita crime rate by town |
| ZN               | proportion of residential land zoned for lots over 25,000 sq.ft. |
| INDUS            | proportion of non-retail business acres per town. |
| NOX              | nitric oxides concentration (parts per 10 million) |
| RM               | average number of rooms per dwelling |
| DIS              | weighted distances to five Boston employment centres |
| RAD              | index of accessibility to radial highways |
| TAX              | full-value property-tax rate per \$10,000 |
| PTRATIO          | pupil-teacher ratio by town |



In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas

boston_data = np.load('data/boston_house_price.npz')
X_train = boston_data['X']
y_train = boston_data['y']

The solution of the linear regression can be written as 
\begin{align}
\mathbf{w} = \mathbf{X}^{+}\mathbf{y}\;,
\end{align}
where $\mathbf{X}^{+}$ is the pseudoinverse of $\mathbf{X}$. 
Recall that $\mathbf{X}$ is a matrix where each row is one sample and $\boldsymbol{y}$
is a vector of target values.



Using the Numpy pseudoinverse function, we will define a 
function to fit a linear regression model to our data. 
If we need to add a bias, we csan simply augment our data.

In [3]:
def fit_LinearRegression(X, y, bias=True):

    # Insert constant ones for bias weights
    if (bias):
        X = np.insert(X, 0, 1, axis=1)

    w = np.linalg.pinv(X).dot(y)
    return w


We define another function for prediction $\hat{y} = \mathbf{w}^\top\mathbf{x}$. Our function below will also compute the mean square error defined as
\begin{align}
\mathcal{L}_{SE} = \frac{1}{m} \sum_{i=1}^m \|y_i - \hat{y}_i\|^2
\end{align}


In [4]:
def predict_LinearRegression(w,X, y, bias=True):

    # Insert constant ones for bias weights
    
    if (bias):
        b_star = w[0]  #recall that with bias, we put the bias into teh first element of w (index = 0) 
        w_star = w[1:] 
        y_hat = X.dot(w_star) + b_star
    else:
        y_hat = X.dot(w)
            
    
    mse = np.mean(np.square(y-y_hat))
    return y_hat, mse    


We first fit a linear model without bias and evaluate it as follows

In [5]:
w1 = fit_LinearRegression(X_train, y_train, bias=False)
y_hat1, mse1 = predict_LinearRegression(w1,X_train, y_train, bias=False)


We then fit a linear model with bias and evaluate it as follows

In [6]:
w2 = fit_LinearRegression(X_train, y_train, bias=True)
y_hat2, mse2 = predict_LinearRegression(w2,X_train, y_train, bias=True)
print("Regression Error: without bias -> {0:0.3f}, with bias -> {1:0.3f}".format(mse1,mse2))

Regression Error: without bias -> 24.166, with bias -> 21.895


As we can see, the model with bias has a lower error. Let's check some of the values

In [7]:
df = pandas.DataFrame({"y": y_train[:5], "without bias": y_hat1[:5], "with bias": y_hat2[:5]})
pandas.set_option('colheader_justify', 'center')
print(df)

     y   without bias  with bias
0  24.0    29.098264   30.003843
1  21.6    24.502275   25.025562
2  34.7    31.227426   30.567597
3  33.4    29.707104   28.607036
4  36.2    29.564796   27.943524


seems that for the very first five samples, the model without bias works better. Let's check out some other samples.

In [8]:
df = pandas.DataFrame({"y": y_train[5:10], "without bias": y_hat1[:5], "with bias": y_hat2[5:10]})
pandas.set_option('colheader_justify', 'center')
print(df)

     y   without bias  with bias
0  28.7    29.098264   25.256284
1  22.9    24.502275   23.001808
2  27.1    31.227426   19.535988
3  16.5    29.707104   11.523637
4  18.9    29.564796   18.920262


Hmm, look, in some cases the model with bias does a much better job. 
Now, let us have some fun. Let us add some non-linear features to our model. The idea folows the concept of polynomial regression, check this [Wikipedia page](https://en.wikipedia.org/wiki/Polynomial_regression) for a brif inroduction

In [9]:
X_quad = np.concatenate((X_train,X_train**2),axis=1)
w3 = fit_LinearRegression(X_quad, y_train, bias=True)
y_hat3, mse3 = predict_LinearRegression(w3,X_quad, y_train, bias=True)
print("Regression Error: with poly-2 features -> {0:0.3f}".format(mse3))


Regression Error: with poly-2 features -> 14.247


In [10]:
df = pandas.DataFrame({"y": y_train[:5], "linear features": y_hat2[:5], "poly-2 features": y_hat3[:5]})
pandas.set_option('colheader_justify', 'center')
print(df)

     y   linear features  poly-2 features
0  24.0     30.003843        28.340697   
1  21.6     25.025562        23.296078   
2  34.7     30.567597        31.926581   
3  33.4     28.607036        32.024942   
4  36.2     27.943524        30.071343   


Should we continue?

In [11]:
X_cube = np.concatenate((X_train,X_train**2,X_train**3),axis=1)
w4 = fit_LinearRegression(X_cube, y_train, bias=True)
y_hat4, mse4 = predict_LinearRegression(w4,X_cube, y_train, bias=True)
print("Regression Error: with poly-3 features -> {0:0.3f}".format(mse4))
df = pandas.DataFrame({"y": y_train[:5], "linear features": y_hat2[:5], "poly-3 features": y_hat3[:5]})
pandas.set_option('colheader_justify', 'center')
print(df)


Regression Error: with poly-3 features -> 13.061
     y   linear features  poly-3 features
0  24.0     30.003843        28.340697   
1  21.6     25.025562        23.296078   
2  34.7     30.567597        31.926581   
3  33.4     28.607036        32.024942   
4  36.2     27.943524        30.071343   


Now, the beauty is that with neural network, we will learn how to extract non-linear features that are desirable for the problem in hand (instead of engineering them).